In [2]:
import numpy
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_text as tft
import tensorflow_hub

In [3]:
def clean_df(df):
    df = df.drop(['q_title', 'q_id', 'line_id'], axis=1)
    return df

train_src = "./codenrock_ds/TextGenerationDetection/train.csv"
test_src = "./codenrock_ds/TextGenerationDetection/test.csv"
df = pd.read_csv(train_src)
testing_df = pd.read_csv(test_src)

df = clean_df(df)
testing_df = clean_df(testing_df)
train, valid, test = numpy.split(df.sample(frac=1), [int(0.6 * len(df)), int(0.8 * len(df))])
train.head()


d:\progdata\anaconda\envs\djangoNS\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,label,ans_text
3816,ai_answer,"Привет! Я бы посоветовал прочитать книгу ""Моск..."
2920,hu_answer,Про идентичность. )\nЯ как-то проектировала в ...
272,hu_answer,By manually computing the similarity and playi...
4397,ai_answer,LINQ methods have different run-time complexit...
3146,ai_answer,"Yes, it is possible to use a Twitter timeline ..."


In [4]:
def scale_dataset(data):
    data["label"] = (data["label"] == "hu_answer").astype(int)
    y = data[data.columns[0]].values
    X = data.drop(data.columns[0], axis=1).values
    data = numpy.hstack((X, numpy.reshape(y, (-1, 1))))
    
    return data, X, y

In [5]:
train_data, X_train, y_train = scale_dataset(train)
valid_data, X_valid, y_valid = scale_dataset(valid)
test_data, X_test, y_test = scale_dataset(train)

In [6]:
bert_preprocess = tensorflow_hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = tensorflow_hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [7]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

layer = tf.keras.layers.Dropout(0.1, name='dropout')(outputs['pooled_output'])
layer = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(layer)

model = tf.keras.Model(inputs=[text_input], outputs=[layer])


In [10]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics='accuracy'
)

In [11]:
model.fit(X_train, y_train, epochs=12)
val_loss = model.evaluate(X_valid, y_valid)[0]

Epoch 1/12
91/91 [==============================] - 747s 8s/step - loss: 0.6531 - accuracy: 0.6012
Epoch 2/12
91/91 [==============================] - 714s 8s/step - loss: 0.6192 - accuracy: 0.6461
Epoch 3/12
91/91 [==============================] - 714s 8s/step - loss: 0.6094 - accuracy: 0.6444
Epoch 4/12
91/91 [==============================] - 723s 8s/step - loss: 0.5953 - accuracy: 0.6582
Epoch 5/12
91/91 [==============================] - 716s 8s/step - loss: 0.5825 - accuracy: 0.6759
Epoch 6/12
91/91 [==============================] - 717s 8s/step - loss: 0.5830 - accuracy: 0.6717
Epoch 7/12
91/91 [==============================] - 712s 8s/step - loss: 0.5696 - accuracy: 0.6825
Epoch 8/12
91/91 [==============================] - 712s 8s/step - loss: 0.5605 - accuracy: 0.6998
Epoch 9/12
91/91 [==============================] - 713s 8s/step - loss: 0.5642 - accuracy: 0.6918
Epoch 10/12
91/91 [==============================] - 714s 8s/step - loss: 0.5467 - accuracy: 0.7053
Epoch 11/

In [12]:
model.save('saved_model/')

INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


In [14]:
test = model.predict(["take ur pants up"])

1/1 [==============================] - 7s 7s/step


In [15]:
test

array([[0.46780157]], dtype=float32)

In [16]:
test_gpt = model.predict(["Лучшее время для посадки картофеля зависит от нескольких факторов:Сорт картофеля. Ранние сорта сажают в первых числах мая, среднеспелые - в середине мая. Поздние сорта имеют более высокую урожайностьРегион выращивания. В южных регионах картофель можно высаживать после 25 апреля, а максимально поздняя дата посадки - 10 июня. В более холодных регионах картофель сажают летом.Температура почвы.Сажать картофель следует, когда температура почвы на глубине 10-12 см составляет 7-8°C, что обычно бывает при установлении среднесуточной температуры воздуха выше 8°CТаким образом, оптимальное время для посадки картофеля в средней полосе России - это первая-вторая декада мая, когда почва прогрелась до необходимой температуры и нет угрозы заморозков. Ранние сорта можно сажать в конце апреля, а поздние - до середины июня"])


1/1 [==============================] - 1s 629ms/step


In [17]:
test_gpt

array([[0.4607154]], dtype=float32)

In [18]:
test_human = model.predict(["Здраств-те! Всю жизнь сажали в конце мая, а до 1 сентября выкапывали. Так, а что не нравится в посадке в апреле-мае? Почему сажать в июне, больше плюсов? Может лучше урожай?"])

1/1 [==============================] - 1s 637ms/step


In [19]:
test_human

array([[0.33273706]], dtype=float32)

In [20]:
another_test = model.predict(["Лучшее время для посадки картофеля зависит от региона и климатических условий, но в целом оптимальным периодом считается середина-конец мая. В средней полосе России картофель обычно сажают в конце мая, когда почва прогреется до 8-10°C на глубине 10 см. Ранняя посадка в апреле-начале мая возможна, но требует укрытия всходов от возвратных заморозков. Поздняя посадка в июне-июле дает меньший урожай, так как растения не успевают сформировать клубни до наступления холодов. Для получения раннего урожая можно использовать ранние сорта и высаживать их в апреле под временные укрытия. Оптимальная глубина посадки - 8-10 см, расстояние между клубнями - 30-35 см, между рядами - 60-70 см. Соблюдение этих рекомендаций позволит получить хороший урожай картофеля."])

1/1 [==============================] - 1s 875ms/step


In [21]:
another_test

array([[0.55117077]], dtype=float32)

In [52]:
yet_another_q = model.predict(["я искуственный интеллект"])

1/1 [==============================] - 1s 584ms/step


In [53]:
yet_another_q

array([[0.57606775]], dtype=float32)

In [40]:
test_data

array([["Well, this is a pretty basic question. You can add two numbers without using any arithmetic operator by using bitwise operators. Here's how you can do it:\n\nint add(int x, int y)\n{\n    while (y != 0)\n    {\n        int carry = x & y;\n        x = x ^ y;\n        y = carry << 1;\n    }\n    return x;\n}\n\nThis code uses the bitwise AND operator to calculate the carry and the bitwise XOR operator to add the two numbers. The carry is then shifted to the left by one bit and added to the sum. This process is repeated until there is no carry left.\n\nSo, there you have it. A simple solution to add two numbers without using any arithmetic operator.",
        0],
       ["This is a counting problem, not a construction problem, so we can approach it using recursion. Since the problem has two natural parts, looking from the left and looking from the right, break it up and solve for just one part first.\nLet b(N, L, R)\nbe the number of solutions, and let f(N, L)\nbe the number of a

1/1 [==============================] - 1s 741ms/step


In [41]:
model.fit(X_train, y_train, epochs=2)

Epoch 1/2
91/91 [==============================] - 811s 9s/step - loss: 0.5439 - accuracy: 0.7046
Epoch 2/2
91/91 [==============================] - 789s 9s/step - loss: 0.5349 - accuracy: 0.7112


In [56]:
model.save('newmodelgptohu/')

INFO:tensorflow:Assets written to: newmodelgptohu/assets


INFO:tensorflow:Assets written to: newmodelgptohu/assets


In [9]:
model = tf.keras.models.load_model('newmodelgptohu')

In [10]:
from sklearn.metrics import classification_report
y_train_predict = model.predict(X_train)

10/91 [==>...........................] - ETA: 15:07

In [ ]:
classification_report(y_train_predict, y_train)
